In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Consumo de Cimento (t)
0,2003-1,0.695898,0.330279,8.630942,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,51.313080,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,28.193
1,2003-2,0.696043,0.331137,8.624872,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,51.554195,4.760190e+07,2.521321e+06,12.492164,4.320154e+07,31.226
2,2003-3,0.696187,0.331306,8.618803,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,51.523306,4.765911e+07,2.522846e+06,12.493707,4.321789e+07,33.484
3,2003-4,0.696332,0.331791,8.612734,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,51.639628,4.771632e+07,2.524370e+06,12.495250,4.323425e+07,31.596
4,2003-5,0.696476,0.332189,8.606665,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,51.838456,4.777353e+07,2.525895e+06,12.496793,4.325060e+07,32.919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.746275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.352061,NaN,NaN,NaN,NaN,66.077
236,2022-9,NaN,0.746582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.160630,NaN,NaN,NaN,NaN,58.590
237,2022-10,NaN,0.745406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.937187,NaN,NaN,NaN,NaN,58.793
238,2022-11,NaN,0.742703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.889598,NaN,NaN,NaN,NaN,53.509


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,-2.017357,-1.841109,-0.841381,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.246822,-1.725325,0.115495,1.032096,-1.955531
1,-1.988164,-1.831498,-0.844328,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.217639,-1.704856,0.145069,1.057757,-1.907364
2,-1.958970,-1.829607,-0.847276,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.221378,-1.684387,0.174644,1.083417,-1.859197
3,-1.929776,-1.824180,-0.850223,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.207299,-1.663918,0.204218,1.109077,-1.811030
4,-1.900583,-1.819720,-0.853171,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.183234,-1.643449,0.233793,1.134738,-1.762863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.146285,0.624622,1.092028,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-2.063199,1.193835,-1.214518,-0.726615,1.324047
188,1.136919,0.664868,1.088543,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-2.019550,1.186279,-1.186673,-0.718777,1.320189
189,1.127552,0.713988,1.085058,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.981485,1.178724,-1.158827,-0.710938,1.316331
190,1.118186,0.765366,1.081573,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.940641,1.171169,-1.130982,-0.703100,1.312472


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amazonas - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,-2.017357,-1.841109,-0.841381,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.246822,-1.725325,0.115495,1.032096,-1.955531
1,-1.988164,-1.831498,-0.844328,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.217639,-1.704856,0.145069,1.057757,-1.907364
2,-1.958970,-1.829607,-0.847276,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.221378,-1.684387,0.174644,1.083417,-1.859197
3,-1.929776,-1.824180,-0.850223,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.207299,-1.663918,0.204218,1.109077,-1.811030
4,-1.900583,-1.819720,-0.853171,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.183234,-1.643449,0.233793,1.134738,-1.762863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.448925,0.452650,1.390527,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.823205,1.169567,-1.503954,-1.462487,0.955680
158,1.452046,0.404414,1.372620,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.905554,1.175278,-1.511151,-1.435811,0.975593
159,1.455166,0.365711,1.354712,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.983877,1.180989,-1.518349,-1.409135,0.995507
160,1.458287,0.316318,1.336804,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-1.057189,1.186700,-1.525547,-1.382459,1.015420


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
157    44.068
158    62.847
159    30.374
160    60.003
161    81.014
Name: Amazonas - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
162,1.464528,0.300771,1.300989,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-1.196117,1.198122,-1.539943,-1.329107,1.055246
163,1.467649,0.302678,1.283081,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-1.258972,1.203832,-1.547140,-1.302431,1.075160
164,1.470769,0.311080,1.265173,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-1.316525,1.209543,-1.554338,-1.275755,1.095073
165,1.473890,0.323489,1.247265,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-1.376631,1.215254,-1.561536,-1.249079,1.114986
166,1.477010,0.339232,1.229358,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-1.427803,1.220965,-1.568734,-1.222403,1.134899
167,1.480131,0.356121,1.211450,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-1.492489,1.226676,-1.575932,-1.195727,1.154813
168,1.483251,0.375580,1.193542,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-1.557787,1.232387,-1.583129,-1.169051,1.174726
169,1.460634,0.394947,1.187115,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.627941,1.233581,-1.568655,-1.136753,1.189420
170,1.438018,0.409500,1.180689,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.689803,1.234776,-1.554180,-1.104456,1.204114
171,1.415401,0.414230,1.174262,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.759409,1.235970,-1.539706,-1.072159,1.218808


In [27]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    64.862
163    49.885
164    53.130
165    57.796
166    47.026
167    74.919
168    34.733
169    15.022
170    20.879
171    36.409
172    22.277
173    20.529
174    37.061
175    23.387
176    38.399
177    39.652
178    40.204
179    21.625
180    42.731
181    36.026
182    34.609
183    40.787
184    39.823
185    39.957
186    52.048
187    53.567
188    46.239
189    51.066
190    51.684
191    44.907
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [28]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [29]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [30]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [31]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[806066579, 2049285293, 2032192701, 4022980133, 153679890, 3748899833, 2641817026, 924203712, 3049072839, 909227647, 3144488690, 3978828576, 671243646, 602837307, 1492069559, 703063686, 3253050182, 2948163872, 4108259500, 1251389346, 3694369386, 590851820, 1116096677, 3826407799, 313085354]


Step: 0 ___________________________________________
val_loss: 403.24072265625
winner_seed: 806066579


Step: 1 ___________________________________________
val_loss: 2474.42578125


Step: 2 ___________________________________________
val_loss: 2365.111328125


Step: 3 ___________________________________________
val_loss: 2836.40869140625


Step: 4 ___________________________________________
val_loss: 229.6756134033203
winner_seed: 153679890


Step: 5 ___________________________________________
val_loss: 907.9139404296875


Step: 6 ___________________________________________
val_loss: 1247.163818359375


Step: 7 ___________________________________________
val_loss: 2203.541015625


Step: 8 _________

In [32]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 3849.3777 - val_loss: 136.9435
Epoch 2/10000
5/5 [==============================] - 0s 7ms/step - loss: 2914.7346 - val_loss: 248.1460
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 2079.9385 - val_loss: 4742.7827
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 161.6531 - val_loss: 6176.8560
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 138.7357 - val_loss: 6939.2085
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 127.8624 - val_loss: 6769.6777
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 109.4498 - val_loss: 4997.5713
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 123.6347 - val_loss: 4740.0620
Epoch 9/10000
5/5 [==============================] - 0s 6ms/step - loss: 118.2824 - val_loss: 4936.6299
Epoch 10/10000
5/5 [==============================] - 0s 6ms/s

5/5 [==============================] - 0s 7ms/step - loss: 145.3287 - val_loss: 1895.9761
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 120.6947 - val_loss: 2371.4526
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 109.3223 - val_loss: 2826.7202
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 95.6572 - val_loss: 2372.3679
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 103.4437 - val_loss: 2429.3083
Epoch 84/10000
5/5 [==============================] - 0s 6ms/step - loss: 109.4791 - val_loss: 2449.1946
Epoch 85/10000
5/5 [==============================] - 0s 6ms/step - loss: 114.3473 - val_loss: 2674.2302
Epoch 86/10000
5/5 [==============================] - 0s 7ms/step - loss: 109.0487 - val_loss: 2395.4790
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 101.9552 - val_loss: 2809.1545
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - los

Epoch 157/10000
5/5 [==============================] - 0s 6ms/step - loss: 97.0253 - val_loss: 3173.3274
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 104.2195 - val_loss: 3110.0198
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 98.7383 - val_loss: 3315.2231
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 106.3582 - val_loss: 3432.3547
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 95.0511 - val_loss: 3689.3926
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 108.6856 - val_loss: 3746.5657
Epoch 163/10000
5/5 [==============================] - 0s 7ms/step - loss: 115.1830 - val_loss: 3322.9751
Epoch 164/10000
5/5 [==============================] - 0s 6ms/step - loss: 101.5444 - val_loss: 3563.6001
Epoch 165/10000
5/5 [==============================] - 0s 7ms/step - loss: 106.0244 - val_loss: 3420.8525
Epoch 166/10000
5/5 [============================

Epoch 312/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.8796 - val_loss: 3999.8103
Epoch 313/10000
5/5 [==============================] - 0s 7ms/step - loss: 101.9771 - val_loss: 4490.8711
Epoch 314/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.0163 - val_loss: 4539.1914
Epoch 315/10000
5/5 [==============================] - 0s 8ms/step - loss: 98.5390 - val_loss: 4042.8843
Epoch 316/10000
5/5 [==============================] - 0s 7ms/step - loss: 103.9727 - val_loss: 4133.1040
Epoch 317/10000
5/5 [==============================] - 0s 6ms/step - loss: 101.4091 - val_loss: 4633.0234
Epoch 318/10000
5/5 [==============================] - 0s 7ms/step - loss: 102.7352 - val_loss: 4351.1753
Epoch 319/10000
5/5 [==============================] - 0s 7ms/step - loss: 93.9848 - val_loss: 4237.2930
Epoch 320/10000
5/5 [==============================] - 0s 7ms/step - loss: 91.5982 - val_loss: 4231.3721
Epoch 321/10000
5/5 [==============================

Epoch 390/10000
5/5 [==============================] - 0s 7ms/step - loss: 92.4111 - val_loss: 3028.2231
Epoch 391/10000
5/5 [==============================] - 0s 7ms/step - loss: 90.6084 - val_loss: 3191.4304
Epoch 392/10000
5/5 [==============================] - 0s 7ms/step - loss: 103.9041 - val_loss: 3001.6357
Epoch 393/10000
5/5 [==============================] - 0s 8ms/step - loss: 96.8969 - val_loss: 2832.4233
Epoch 394/10000
5/5 [==============================] - 0s 6ms/step - loss: 96.1848 - val_loss: 3091.1257
Epoch 395/10000
5/5 [==============================] - 0s 7ms/step - loss: 94.7714 - val_loss: 3565.3311
Epoch 396/10000
5/5 [==============================] - 0s 7ms/step - loss: 98.7705 - val_loss: 2901.5566
Epoch 397/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.6244 - val_loss: 3119.9365
Epoch 398/10000
5/5 [==============================] - 0s 7ms/step - loss: 88.6050 - val_loss: 3727.2322
Epoch 399/10000
5/5 [==============================] -

...

5/5 [==============================] - 0s 7ms/step - loss: 93.1476 - val_loss: 3297.3862
Epoch 1630/10000
5/5 [==============================] - 0s 6ms/step - loss: 95.3772 - val_loss: 3492.9624
Epoch 1631/10000
5/5 [==============================] - 0s 7ms/step - loss: 93.3172 - val_loss: 3293.7734
Epoch 1632/10000
5/5 [==============================] - 0s 8ms/step - loss: 96.9631 - val_loss: 3769.2944
Epoch 1633/10000
5/5 [==============================] - 0s 8ms/step - loss: 90.6304 - val_loss: 3652.6848
Epoch 1634/10000
5/5 [==============================] - 0s 8ms/step - loss: 93.0667 - val_loss: 2959.3418
Epoch 1635/10000
5/5 [==============================] - 0s 7ms/step - loss: 98.8275 - val_loss: 3853.4463
Epoch 1636/10000
5/5 [==============================] - 0s 8ms/step - loss: 91.6672 - val_loss: 3351.9219
Epoch 1637/10000
5/5 [==============================] - 0s 9ms/step - loss: 103.2920 - val_loss: 2451.3984
Epoch 1638/10000
5/5 [==============================] - 0s 7ms

5/5 [==============================] - 0s 7ms/step - loss: 108.0649 - val_loss: 2560.0959
Epoch 1784/10000
5/5 [==============================] - 0s 6ms/step - loss: 98.9343 - val_loss: 2629.6863
Epoch 1785/10000
5/5 [==============================] - 0s 7ms/step - loss: 109.8708 - val_loss: 2342.2844
Epoch 1786/10000
5/5 [==============================] - 0s 7ms/step - loss: 95.7266 - val_loss: 2491.5681
Epoch 1787/10000
5/5 [==============================] - 0s 11ms/step - loss: 92.9974 - val_loss: 2500.2083
Epoch 1788/10000
5/5 [==============================] - 0s 8ms/step - loss: 96.0029 - val_loss: 2755.8035
Epoch 1789/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.8855 - val_loss: 3061.4805
Epoch 1790/10000
5/5 [==============================] - 0s 7ms/step - loss: 119.8750 - val_loss: 3360.0325
Epoch 1791/10000
5/5 [==============================] - 0s 8ms/step - loss: 102.8025 - val_loss: 3519.5381
Epoch 1792/10000
5/5 [==============================] - 0s

5/5 [==============================] - 0s 7ms/step - loss: 95.5244 - val_loss: 3337.2666
Epoch 1861/10000
5/5 [==============================] - 0s 8ms/step - loss: 96.6815 - val_loss: 3651.4744
Epoch 1862/10000
5/5 [==============================] - 0s 8ms/step - loss: 107.2639 - val_loss: 3288.8193
Epoch 1863/10000
5/5 [==============================] - 0s 7ms/step - loss: 102.8922 - val_loss: 3086.2114
Epoch 1864/10000
5/5 [==============================] - 0s 7ms/step - loss: 91.4062 - val_loss: 3211.5742
Epoch 1865/10000
5/5 [==============================] - 0s 7ms/step - loss: 97.9998 - val_loss: 3207.2366
Epoch 1866/10000
5/5 [==============================] - 0s 7ms/step - loss: 94.1031 - val_loss: 3119.3210
Epoch 1867/10000
5/5 [==============================] - 0s 8ms/step - loss: 93.2032 - val_loss: 3197.8115
Epoch 1868/10000
5/5 [==============================] - 0s 7ms/step - loss: 102.7644 - val_loss: 3552.8347
Epoch 1869/10000
5/5 [==============================] - 0s 7

5/5 [==============================] - 0s 7ms/step - loss: 89.2112 - val_loss: 2815.6426
Epoch 1938/10000
5/5 [==============================] - 0s 7ms/step - loss: 82.4156 - val_loss: 3060.7104
Epoch 1939/10000
5/5 [==============================] - 0s 7ms/step - loss: 90.8070 - val_loss: 3086.0161
Epoch 1940/10000
5/5 [==============================] - 0s 7ms/step - loss: 98.7452 - val_loss: 3211.3997
Epoch 1941/10000
5/5 [==============================] - 0s 7ms/step - loss: 101.9325 - val_loss: 3136.5000
Epoch 1942/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.8059 - val_loss: 2988.0413
Epoch 1943/10000
5/5 [==============================] - 0s 7ms/step - loss: 114.0226 - val_loss: 3263.2883
Epoch 1944/10000
5/5 [==============================] - 0s 8ms/step - loss: 118.8189 - val_loss: 3182.6926
Epoch 1945/10000
5/5 [==============================] - 0s 7ms/step - loss: 123.3176 - val_loss: 3516.4595
Epoch 1946/10000
5/5 [==============================] - 0s 

Epoch 2014/10000
5/5 [==============================] - 0s 8ms/step - loss: 93.7933 - val_loss: 3292.1121
Epoch 2015/10000
5/5 [==============================] - 0s 7ms/step - loss: 97.8227 - val_loss: 3017.6448
Epoch 2016/10000
5/5 [==============================] - 0s 8ms/step - loss: 103.3157 - val_loss: 3603.9136
Epoch 2017/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.6744 - val_loss: 3269.6282
Epoch 2018/10000
5/5 [==============================] - 0s 7ms/step - loss: 91.6627 - val_loss: 3005.4636
Epoch 2019/10000
5/5 [==============================] - 0s 7ms/step - loss: 94.4623 - val_loss: 3038.5583
Epoch 2020/10000
5/5 [==============================] - 0s 7ms/step - loss: 98.6087 - val_loss: 3136.2942
Epoch 2021/10000
5/5 [==============================] - 0s 7ms/step - loss: 91.7410 - val_loss: 2754.2693
Epoch 2022/10000
5/5 [==============================] - 0s 7ms/step - loss: 90.5432 - val_loss: 2940.3293
Epoch 2023/10000
5/5 [=======================

In [33]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [34]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,32.639999,32.19994,31.464287,30.929655,29.465794,28.594357,28.273865,27.558643,27.338331,26.696087,26.185539,25.414291,25.501762,25.354351,25.181099,25.126312,25.101665,25.119028,25.1117,25.110744,25.122297,25.127419,25.132656,25.168053,25.164726,25.206028,25.289688,25.424046,25.729954,27.051828
Target,64.862,49.885,53.13,57.796,47.026,74.919,34.733,15.022,20.879,36.409,22.277,20.529,37.061,23.387,38.399,39.652,40.204,21.625,42.731,36.026,34.609,40.787,39.823,39.957,52.048,53.567,46.239,51.066,51.684,44.907
Error,32.222,17.685059,21.665714,26.866346,17.560207,46.324642,6.459137,12.536643,6.459332,9.712914,3.908539,4.885292,11.559238,1.967352,13.217899,14.525688,15.102333,3.494028,17.619299,10.915257,9.486704,15.65958,14.690346,14.788948,26.883274,28.360973,20.94931,25.641956,25.954044,17.855173


In [35]:
display(mae)
display(mape)

16.498573

0.37942818

In [36]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [37]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[346.7608]] - Target[497.4669999999999]| =  Error: [[150.7062]]; MAPE:[[0.30294713]]
1/1 [==============================] - 0s 19ms/step
Ano-0: |Prediction[[302.1571]] - Target[434.26099999999997]| =  Error: [[132.10388]]; MAPE:[[0.3042039]]
1/1 [==============================] - 0s 18ms/step
Ano-5: |Prediction[[153.86627]] - Target[299.511]| =  Error: [[145.64471]]; MAPE:[[0.48627502]]


[array([[150.7062]], dtype=float32),
 array([[132.10388]], dtype=float32),
 array([[145.64471]], dtype=float32)]

142.81827

0.36447534